In [19]:
import requests
from pydantic import BaseModel
from typing import List, Literal
from geopy.geocoders import Nominatim
import folium
from folium import Marker, Circle

In [38]:
class Ambulatorio(BaseModel) :
    orario: str 
    telefono: str 
    segreteria: str
    comune: str
    ubicazione: str
    principale: Literal["Y", "N"]


class Doc(BaseModel) :
     nome: str 
     cognome: str
     tipologia: Literal["PLS","MMG"]
     ambito: str
     aft: str
     codice_regionale: str
     scelte_disponibili: int
     id: str
     ambulatori : List[Ambulatorio]
     
class Location(BaseModel) :
    latitude : float
    longitude : float

In [6]:
url_prova = "https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&azienda=1813960"
url_prova 

'https://servizi.estar.toscana.it/adiba/ambulatori.php?a=0&azienda=1813960'

In [7]:
req  = requests.get(url_prova)
req

<Response [200]>

In [8]:
json_req = req.json()
json_req["medici"]

[{'nome': 'CARMELA',
  'cognome': 'ABELA',
  'tipologia': 'MMG',
  'ambito': 'FIRENZE - GEN',
  'aft': 'CAMPO DI MARTE',
  'codice_regionale': '010165',
  'scelte_disponibili': '1109',
  'id': '10010665',
  'ambulatori': [{'orario': 'Lun : 16:00 - 17:00(A); Mer : 15:30 - 16:30(A); Ven : 10:00 - 11:00(A); ',
    'telefono': '3206978229',
    'segreteria': '3206978229   3206978229',
    'comune': 'FIRENZE',
    'ubicazione': 'VIA SANTO STEFANO IN PANE, 4/A, FIRENZE',
    'principale': 'N'},
   {'orario': 'Lun : 18:30 - 19:30(A); Mar : 09:00 - 11:00(A); Gio : 09:00 - 10:00(A); ',
    'telefono': '3206978229',
    'segreteria': '3206978229   3206978229',
    'comune': 'FIRENZE',
    'ubicazione': 'VIA DI CAMERATA, 10, FIRENZE',
    'principale': 'N'}]},
 {'nome': 'FIAMMA',
  'cognome': 'ABRAMI',
  'tipologia': 'MMG',
  'ambito': 'DICOMANO-LONDA-SAN GODENZO - GEN',
  'aft': 'MUGELLO EST',
  'codice_regionale': '000455',
  'scelte_disponibili': '122',
  'id': '10017279',
  'ambulatori': [{'o

In [30]:
def get_pediatri(lista_medici : list,)  -> List[Doc] :
    pediatri = []
    for medico in lista_medici :
        #print(medico["tipologia"])
        if medico["tipologia"] == "PLS" :
            pediatri.append(Doc(**medico))
    return pediatri
     

In [31]:
pediatri = get_pediatri(json_req["medici"])

In [51]:
pediatri[0].ambulatori

[Ambulatorio(orario='Lun Mer Gio : 15:30 - 19:30(A); Mar Ven : 10:00 - 13:00(A); ', telefono='055570017', segreteria='055570017', comune='FIRENZE', ubicazione='VIA BORGHINI, 1, FIRENZE', principale='Y'),
 Ambulatorio(orario='Lun Mer Gio : 08:00 - 12:00; Mar Ven : 08:00 - 10:00; 14:30 - 16:30; ', telefono='366-3775315', segreteria='366-3775315', comune='FIRENZE', ubicazione="CONTATTABILITA' TELEFONICA, FIRENZE", principale='N')]

In [48]:
def get_lat_log_from_address(address :str, geolocator : object) -> Location :
    location = geolocator.geocode(address)
    return Location(latitude=location.latitude, longitude=location.longitude)

In [46]:
def add_marker_to_map(lista_medici :List[Doc], mappa_base : folium.Map) :
    geocoder = Nominatim(user_agent="asl_ricerca")
    for medici in lista_medici :
        lista_unbicazioni = medici.ambulatori
        
        location = get_lat_log_from_address(medici["ubicazione"], geocoder)
        merk = Marker([location.latitude, location.longitude],).add_to(mappa_base)
    return mappa_base   
    

In [10]:
prova_ambulatorio = json_req["medici"][0]["ambulatori"][0]
prova_ambulatorio

{'orario': 'Lun : 16:00 - 17:00(A); Mer : 15:30 - 16:30(A); Ven : 10:00 - 11:00(A); ',
 'telefono': '3206978229',
 'segreteria': '3206978229   3206978229',
 'comune': 'FIRENZE',
 'ubicazione': 'VIA SANTO STEFANO IN PANE, 4/A, FIRENZE',
 'principale': 'N'}

In [14]:
geolocator = Nominatim(user_agent="asl_ricerca")
location_prova = geolocator.geocode(prova_ambulatorio["ubicazione"])
print(location_prova.address)

4/A R, Via Santo Stefano in Pane, Rifredi, Quartiere 5, Firenze, Toscana, 50134, Italia


In [37]:
location_prova.attribute

AttributeError: 'Location' object has no attribute 'attribute'

In [45]:
mappa_base = folium.Map(location=[43.76,11.25], zoom_start=13)
mappa_base

In [20]:
Marker([location_prova.latitude, location_prova.longitude], ).add_to(mappa_base)

In [49]:
mappa_base = add_marker_to_map(pediatri, mappa_base)
mappa_base

TypeError: 'Doc' object is not subscriptable

In [21]:
mappa_base